In [1]:
import pandas as pd
import vectorbt as vbt
from datetime import datetime
import numpy as np
import glob
import os
import json
import calendar

pd.set_option("future.no_silent_downcasting", True)

Start Dates and End Dates For the Simulation

In [2]:
start_date = "2009-01-01"
end_date = "2024-05-01"

### Marking Days of Trading

* Buy on last trading day of month.
* Sell at the end of third trading day of the next month

In [3]:
df = pd.read_csv(r"data\master\IndiaTradingCalendar.csv")

# Converting Date dtype and creating Month and Year columns
df.Date = pd.to_datetime(df.loc[:, "Date"])
df.loc[:, "Month"] = df.Date.dt.month
df.loc[:, "Year"] = df.Date.dt.year

# subsetting only for trading days and numbering days 1 to n
temp = df[(df["Trading Day"] == 1)].reset_index(drop=True)
temp.loc[:, "TradingDayNum"] = temp.groupby(by=["Month", "Year"]).cumcount() + 1


# Initializing tun of month indicators 1,2,3,4 for the four trading days
# Marking last trading day in the previous month (entry point)
temp.loc[
    temp.index.isin(
        [each for each in temp[temp["TradingDayNum"] == 1].index - 1 if each >= 0]
    ),
    "TurnOfMonthIndicator",
] = 1

# marking first three days of the month
temp.loc[temp["TradingDayNum"] == 1.0, "TurnOfMonthIndicator"] = 2
temp.loc[temp["TradingDayNum"] == 2.0, "TurnOfMonthIndicator"] = 3
temp.loc[temp["TradingDayNum"] == 3.0, "TurnOfMonthIndicator"] = 4

# Merging it with Trading Calendar df
df = df.merge(
    right=temp, how="left", on=["Date", "Day", "Trading Day", "Month", "Year"]
)

# Converting date column after merging to DateTime and localizing the timezone to IST
df.Date = pd.to_datetime(df.loc[:, "Date"])
df.Date = df.Date.dt.tz_localize("UTC+05:30")


# Marking Entry and Exists for the Turn of Month Strategy
df.loc[:, "TurnOfMonthEntry"] = df.loc[:, "TurnOfMonthIndicator"] == 1
df.loc[:, "TurnOfMonthExits"] = df.loc[:, "TurnOfMonthIndicator"] == 4

df.sample(25)

,Date,Day,Trading Day,Month,Year,TradingDayNum,TurnOfMonthIndicator,TurnOfMonthEntry,TurnOfMonthExits
2153,2014-11-24 00:00:00+05:30,Monday,1.0,11,2014,14.0,NaN,False,False
4300,2020-10-10 00:00:00+05:30,Saturday,0.0,10,2020,NaN,NaN,False,False
5726,2024-09-05 00:00:00+05:30,Thursday,1.0,9,2024,4.0,NaN,False,False
1317,2012-08-10 00:00:00+05:30,Friday,1.0,8,2012,8.0,NaN,False,False
5012,2022-09-22 00:00:00+05:30,Thursday,1.0,9,2022,16.0,NaN,False,False
633,2010-09-26 00:00:00+05:30,Sunday,0.0,9,2010,NaN,NaN,False,False
3566,2018-10-07 00:00:00+05:30,Sunday,0.0,10,2018,NaN,NaN,False,False
1750,2013-10-17 00:00:00+05:30,Thursday,1.0,10,2013,11.0,NaN,False,False
1803,2013-12-09 00:00:00+05:30,Monday,1.0,12,2013,6.0,NaN,False,False
5427,2023-11-11 00:00:00+05:30,Saturday,0.0,11,2023,NaN,NaN,False,False


### Reading Special Holidays 
Here data is available but either the day was holiday (where the open and close are basically the same) or was a special trading session (like muhurat trading during Diwali).

In [4]:
# Reading the special holiday file
special_holidays = pd.read_csv("additional_files\SpecialHolidays.csv")
special_holidays.Date = pd.to_datetime(special_holidays.Date)
special_holidays.Date = special_holidays.Date.dt.tz_localize("UTC+05:30")


# Making a list of all holidays
all_holidays = list(special_holidays.Date.dt.date.values) + list(
    df.loc[df["Trading Day"] == 0, "Date"].dt.date.values
)

### Loading Nifty Data

Reading the Nifty Data Created by the `data_collate_nifty50.py`.

In [5]:
nifty_next50 = pd.read_csv(r"data\master\NiftyNext50.csv")

### Loading Delisted Symbols and Name Change Mappings

Some securities have been delisted, renamed or merged, for those securities we created a mapping.

Here the status of being delisted can mean two things:

    1) The company has been delisted.
    2) The company has merged with an other company and is not a actively traded instrument.

In [6]:
delisting_name_mapping = json.load(open("additional_files\delisting_new_name.json"))
delisting_name_mapping

{'LTI': {'Name': 'LTIM.NS', 'Status': 'Ok'},
 'MINDTREE': {'Name': 'MINDTREE.NS', 'Status': 'Delisted'},
 'ADANITRANS': {'Name': 'ADANIENSOL.NS', 'Status': 'Ok'},
 'CADILAHC': {'Name': 'ZYDUSLIFE.NS', 'Status': 'Ok'},
 'MOTHERSUMI': {'Name': 'MOTHERSON.NS', 'Status': 'Ok'},
 'TATAGLOBAL': {'Name': 'TATACONSUM.NS', 'Status': 'Ok'},
 'GSKCONS': {'Name': 'GSKCONS.NS', 'Status': 'Delisted'},
 'SRTRANSFIN': {'Name': 'SHRIRAMFIN.NS', 'Status': 'Ok'},
 'UNIPHOS': {'Name': 'UNIENTER.NS', 'Status': 'Ok'},
 'CROMPGREAV': {'Name': 'CROMPGREAV.NS', 'Status': 'Delisted'},
 'VIJAYABANK': {'Name': 'VIJAYABANK.NS', 'Status': 'Delisted'},
 'SYNDIBANK': {'Name': 'SYNDIBANK.NS', 'Status': 'Delisted'},
 'RNRL': {'Name': 'RNRL.NS', 'Status': 'Delisted'},
 'PATNI': {'Name': 'PATNI.NS', 'Status': 'Delisted'},
 'NIRMA': {'Name': 'NIRMA.NS', 'Status': 'Delisted'},
 'MUNDRAPORT': {'Name': 'ADANIPORTS.NS', 'Status': 'Ok'},
 'INGVYSYABK': {'Name': 'INGVYSYABK.NS', 'Status': 'Delisted'},
 'CORPBANK': {'Name': 'COR

### Preparing Entry and Exits 

Creating a data frame that mark entry and exists for different securities.

In [7]:
TurnOfMonthEntry = pd.DataFrame()
TurnOfMonthExits = pd.DataFrame()

for year in range(2009, datetime.now().year + 1):

    for month in range(1, 13):

        if (year == datetime.now().year) & (month == datetime.now().month):
            break

        month_name = calendar.month_abbr[month]

        # Repeat the column values into 50 columns using NumPy
        repeated_values_entry = np.tile(
            df.loc[
                (df.loc[:, "Year"] == year)
                & (df.loc[:, "Month"] == month)
                & (df.loc[:, "Trading Day"] == 1),
                "TurnOfMonthEntry",
            ].values,
            (50, 1),
        ).T

        repeated_values_exit = np.tile(
            df.loc[
                (df.loc[:, "Year"] == year)
                & (df.loc[:, "Month"] == month)
                & (df.loc[:, "Trading Day"] == 1),
                "TurnOfMonthExits",
            ].values,
            (50, 1),
        ).T

        symbols = nifty_next50.loc[
            (nifty_next50.loc[:, "Year"] == year)
            & (nifty_next50.loc[:, "Month"] == month_name),
            "Security Symbol",
        ].unique()

        symbols = [
            (
                delisting_name_mapping.get(symbol)["Name"]
                if symbol in delisting_name_mapping.keys()
                else symbol + ".NS"
            )
            for symbol in symbols
        ]

        df_repeated_entry = pd.DataFrame(repeated_values_entry, columns=symbols)

        df_repeated_exit = pd.DataFrame(
            repeated_values_exit,
            columns=symbols,
        )

        # Assigning the Date
        df_repeated_entry["Date"] = df.loc[
            (df.Year == year) & (df.Month == month) & (df.loc[:, "Trading Day"] == 1),
            "Date",
        ].dt.date.values

        df_repeated_exit["Date"] = df.loc[
            (df.Year == year) & (df.Month == month) & (df.loc[:, "Trading Day"] == 1),
            "Date",
        ].dt.date.values

        TurnOfMonthEntry = pd.concat(
            [TurnOfMonthEntry, df_repeated_entry], ignore_index=True
        )

        TurnOfMonthExits = pd.concat(
            [TurnOfMonthExits, df_repeated_exit], ignore_index=True
        )


# Filling NA as false because the security was not included in the Nifty Index
TurnOfMonthEntry.fillna(False, inplace=True)
TurnOfMonthExits.fillna(False, inplace=True)

# Converting the date column into a date time index
TurnOfMonthEntry.Date = pd.to_datetime(TurnOfMonthEntry.loc[:, "Date"])
TurnOfMonthEntry.Date = TurnOfMonthEntry.Date.dt.tz_localize("UTC+05:30")

TurnOfMonthExits.Date = pd.to_datetime(TurnOfMonthExits.loc[:, "Date"])
TurnOfMonthExits.Date = TurnOfMonthExits.Date.dt.tz_localize("UTC+05:30")

# Checking to see if all the dates were included
assert (
    df[
        (df.Date < datetime.now().strftime("%Y-%m-01"))
        & (df.loc[:, "Trading Day"] == 1)
    ].shape[0]
    == TurnOfMonthEntry.shape[0]
), "Shape Mismatch: TurnOfMonthEntry"

assert (
    df[
        (df.Date < datetime.now().strftime("%Y-%m-01"))
        & (df.loc[:, "Trading Day"] == 1)
    ].shape[0]
    == TurnOfMonthExits.shape[0]
), "Shape Mismatch: TurnOfMonthExits"

In [8]:
TurnOfMonthEntry.sample(5)

,ABIRLANUVO.NS,ANDHRABANK.NS,APOLLOTYRE.NS,ASHOKLEY.NS,ASIANPAINT.NS,SANOFI.NS,AXISBANK.NS,BANKBARODA.NS,BANKINDIA.NS,BEL.NS,...,ATGL.NS,HAL.NS,IRCTC.NS,AWL.NS,VBL.NS,TVSMOTOR.NS,TRENT.NS,ADANIPOWER.NS,IRFC.NS,JIOFIN.NS
2493,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2628,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
309,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1792,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
TurnOfMonthExits.sample(5)

,ABIRLANUVO.NS,ANDHRABANK.NS,APOLLOTYRE.NS,ASHOKLEY.NS,ASIANPAINT.NS,SANOFI.NS,AXISBANK.NS,BANKBARODA.NS,BANKINDIA.NS,BEL.NS,...,ATGL.NS,HAL.NS,IRCTC.NS,AWL.NS,VBL.NS,TVSMOTOR.NS,TRENT.NS,ADANIPOWER.NS,IRFC.NS,JIOFIN.NS
1501,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
704,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1977,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2138,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
316,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Reading Yahoo Data 



In [10]:
open_price = pd.DataFrame()
close_price = pd.DataFrame()


def read_delisted_data(symbol: str) -> pd.core.frame.DataFrame:
    """
    Reading and processingt the delisted data for a particular symbol from files.

    Args:
        symbol (str): security symbol for the equity.

    Returns:
        pandas.core.frame.DataFrame: prepared data frame for the equity prices.
    """

    # listing the delisted symbol files
    files = [
        name.replace("\\", "/")
        for name in glob.glob(
            f"additional_files\DelistedMissingStocks\{symbol}\*.csv", recursive=True
        )
    ]

    # Reading the files
    df_delisted_data = pd.concat([pd.read_csv(each) for each in files])
    df_delisted_data.columns = [each.strip() for each in df_delisted_data.columns]

    # Standardizing Date Time to represent exactly as Yahoo Finance
    df_delisted_data.Date = pd.to_datetime(df_delisted_data.loc[:, "Date"])
    df_delisted_data.Date = df_delisted_data.Date.dt.tz_localize("UTC+05:30")

    df_delisted_data = df_delisted_data.rename(
        {"OPEN": "Open", "close": "Close"}, axis=1
    )

    # If string will convert into float (problem observed in HDFC delisted Data)
    df_delisted_data.Open = df_delisted_data.Open.apply(
        lambda x: float(x) if "," not in str(x) else float(x.replace(",", ""))
    )

    df_delisted_data.Close = df_delisted_data.Close.apply(
        lambda x: float(x) if "," not in str(x) else float(x.replace(",", ""))
    )

    df_delisted_data.drop_duplicates(subset="Date", inplace=True)

    return df_delisted_data

# For each symbol pulling the yahoo data
for symbol in nifty_next50[nifty_next50["Year"] >= 2009]["Security Symbol"].unique():
    delisted_flag = False
    if symbol in delisting_name_mapping.keys():
        if delisting_name_mapping.get(symbol)["Status"] == "Delisted":

            delisted_flag = True
            delisted_data = read_delisted_data(symbol=symbol)

            symbol = symbol if ".NS" in symbol else symbol + ".NS"

            if open_price.shape[0] == 0:
                open_price = pd.concat(
                    [
                        open_price,
                        delisted_data.loc[:, ["Date", "Open"]].rename(
                            {"Open": symbol}, axis=1
                        ),
                    ]
                )
            else:
                open_price = open_price.merge(
                    right=delisted_data.loc[:, ["Date", "Open"]].rename(
                        {"Open": symbol}, axis=1
                    ),
                    on="Date",
                    how="outer",
                )

            if close_price.shape[0] == 0:
                close_price = pd.concat(
                    [
                        close_price,
                        delisted_data.loc[:, ["Date", "Close"]].rename(
                            {"Close": symbol}, axis=1
                        ),
                    ]
                )
            else:
                close_price = close_price.merge(
                    right=delisted_data.loc[:, ["Date", "Close"]].rename(
                        {"Close": symbol}, axis=1
                    ),
                    on="Date",
                    how="outer",
                )

        else:
            symbol = delisting_name_mapping.get(symbol)["Name"]

    symbol = symbol if ".NS" in symbol else symbol + ".NS"

    if symbol in open_price.columns:
        continue

    if not delisted_flag:

        data = vbt.YFData.download(
            symbol,
            start=start_date,
            end=end_date,
            tz_convert="Asia/Kolkata",
        ).get(["Open", "Close"])

        data = data.reset_index().copy()

        if data[data.Date.dt.time == "00:00:00"].shape[0] == 0:
            data.Date = data.Date.dt.date
            data.Date = pd.to_datetime(data.loc[:, "Date"])
            data.Date = data.Date.dt.tz_localize("UTC+05:30")

        if open_price.shape[0] == 0:
            open_price = pd.concat(
                [
                    open_price,
                    data.loc[:, ["Date", "Open"]].rename({"Open": symbol}, axis=1),
                ]
            )
        else:
            open_price = open_price.merge(
                right=data.loc[:, ["Date", "Open"]].rename({"Open": symbol}, axis=1),
                on="Date",
                how="outer",
            )

        if close_price.shape[0] == 0:
            close_price = pd.concat(
                [
                    close_price,
                    data.loc[:, ["Date", "Close"]].rename({"Close": symbol}, axis=1),
                ]
            )
        else:
            close_price = close_price.merge(
                right=data.loc[:, ["Date", "Close"]].rename({"Close": symbol}, axis=1),
                on="Date",
                how="outer",
            )

# Filtering out all holidays from open price and close price data
open_price = open_price[~open_price.Date.map(lambda x: x.date() in all_holidays)]
close_price = close_price[~close_price.Date.map(lambda x: x.date() in all_holidays)]


# Filtering out data points that are outside start date (including) and end date (not including)
open_price = open_price[
    (open_price["Date"] >= start_date) & (open_price["Date"] < end_date)
]

close_price = close_price[
    (close_price["Date"] >= start_date) & (close_price["Date"] < end_date)
]

Turning Off Entry and Exit during 2019-03-29 and 2019-04-03 respectively as Yahoo Data is absent for 2019-03-29

In [11]:
TurnOfMonthEntry.loc[TurnOfMonthEntry["Date"] == "2019-03-29", :] = (
    TurnOfMonthEntry.loc[TurnOfMonthEntry["Date"] == "2019-03-29", :].replace(
        to_replace=True, value=False
    )
)

TurnOfMonthExits.loc[TurnOfMonthExits["Date"] == "2019-04-03", :] = (
    TurnOfMonthExits.loc[TurnOfMonthExits["Date"] == "2019-04-03", :].replace(
        to_replace=True, value=False
    )
)

Removing data points for the following dates 2019-02-13 and 2019-03-29 as data is absent from yahoo


In [12]:
TurnOfMonthEntry = TurnOfMonthEntry.loc[
    (TurnOfMonthEntry["Date"] != "2019-02-13")
    & (TurnOfMonthEntry["Date"] != "2019-03-29"),
    :,
].copy()

TurnOfMonthExits = TurnOfMonthExits.loc[
    (TurnOfMonthExits["Date"] != "2019-02-13")
    & (TurnOfMonthExits["Date"] != "2019-03-29"),
    :,
].copy()

Removing the same row for Open Price and CLose Price 29th March, 2019, if exists. 

In [13]:
open_price = open_price.loc[
    (open_price["Date"] != "2019-02-13") & (open_price["Date"] != "2019-03-29"),
    :,
].copy()

close_price = open_price.loc[
    (close_price["Date"] != "2019-02-13") & (close_price["Date"] != "2019-03-29"),
    :,
].copy()

if the stock is present in Nifty Next 50 index in the current month and is excluded in the next month, 

due to the way code is authored, the stock's entry is triggered (marked in the TurnOfMonthEntry data frame) 

but its exit is not (exit not marked in TurnOfMonthExits data frame). 

We resolve it further in the following code:

In [14]:
last_entry_trading_window = (
    df.loc[
        (df["Date"] < datetime.now().strftime("%Y-%m-01"))
        & (df["TurnOfMonthEntry"] == True),
        "Date",
    ]
    .to_list()[-1]
    .date()
    .strftime("%Y-%m-%d")
)

for symbol in [each for each in TurnOfMonthEntry.columns if each != "Date"]:

    last_entry = (
        TurnOfMonthEntry.loc[TurnOfMonthEntry.loc[:, symbol] == True, "Date"]
        .to_list()[-1]
        .date()
        .strftime("%Y-%m-%d")
    )

    if (TurnOfMonthExits.loc[TurnOfMonthExits.Date >= last_entry, symbol].sum() == 0
    ) & (last_entry != last_entry_trading_window):

        unmarked_exit = (
            df.loc[
                (df["Date"] >= last_entry) & (df["TurnOfMonthExits"] == True), "Date"
            ]
            .to_list()[0]
            .date()
            .strftime("%Y-%m-%d")
        )
        
        TurnOfMonthExits.loc[TurnOfMonthExits.Date == unmarked_exit, symbol] = True


    elif last_entry == last_entry_trading_window:
        pass

### Preparing Portfolio Weightage 

    * Based on Nifty Next 50 Index 

In [15]:
# Converting Nifty Weightage into float from object dtype
nifty_next50.loc[:, "Weightage"] = nifty_next50["Weightage"].apply(
    lambda x: float(x.replace("%", ""))
)

size_df = pd.DataFrame()

for each_date in df.loc[df.loc[:, "TurnOfMonthEntry"] == True, "Date"]:

    month = calendar.month_abbr[each_date.month]
    year = each_date.year

    weightage_df = nifty_next50.loc[
        (nifty_next50.Year == year) & (nifty_next50.Month == month),
        ["Security Symbol", "Weightage"],
    ]

    weightage_df["Weightage"] = (
        weightage_df["Weightage"] / weightage_df["Weightage"].sum()
    )

    weightage_df = pd.pivot_table(
        data=weightage_df, values="Weightage", columns="Security Symbol"
    )

    weightage_df.columns = [
        (
            delisting_name_mapping[each]["Name"]
            if each in delisting_name_mapping.keys()
            else (each + ".NS")
        )
        for each in weightage_df.columns
    ]

    weightage_df.loc[:, "Date"] = each_date

    size_df = pd.concat([size_df, weightage_df])

size_df.reset_index(drop=True, inplace=True)

size_df = TurnOfMonthEntry.loc[:, ["Date"]].merge(right=size_df, how="left", on="Date")

Checking shape alsong both dimensions date(rows) and security symbol(columns)

In [16]:
assert (
    TurnOfMonthEntry.shape[0]
    == TurnOfMonthExits.shape[0]
    == open_price.shape[0]
    == close_price.shape[0]
    == size_df.shape[0]
), "Shape Mismatch: Dates(rows) across all 4 data frames"

assert (
    TurnOfMonthEntry.shape[1]
    == TurnOfMonthExits.shape[1]
    == open_price.shape[1]
    == close_price.shape[1]
    == size_df.shape[1]
), "Shape Mismatch: Security Symbols(columns) across all 4 data frames"

### Save the Data

In [17]:
path = "./data/simulation_input/niftynext50/"

if not os.path.exists(path):
    os.makedirs(path)

TurnOfMonthEntry.to_csv(path + "master_TurnOfMonthEntry.csv", index=False)
TurnOfMonthExits.to_csv(path + "master_TurnOfMonthExits.csv", index=False)
open_price.to_csv(path + "master_open_price.csv", index=False)
close_price.to_csv(path + "master_close_price.csv", index=False)
size_df.to_csv(path + "master_weightage.csv", index=False)